# Running Plato in a terminal with less trouble

It is strongly recommended and more convenient to run Plato in a terminal, preferably in Visual Studio Code. However, each time when you connect to a remote virtual machine of Colab, most likely it will be a new one and you need to set conda environment again. This notebook can help you avoid some repetitive and tedious settings. 



# First time

When you run Plato on Colab for the first time, please do the following steps.



## 1. Install Plato on Google Drive

Since Google Colab removes all the files that you have downloaded or created when you end a session, the best option is to use GitHub to store your code, and Google Drive to store your datasets, logs, and anything else that would normally reside on your filesystem but wouldn’t be tracked by a git repo.

When you run the code below, you will need to click a link and follow a process that takes a few seconds. When the process is complete, all of your drive files will be available via ‘/root/drive’ on your Colab instance, and this will allow you to structure your projects in the same way you would if you were using a cloud server.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
root_path = '/content/drive/My\ Drive'
%cd $root_path



Clone Plato's public git repository on GitHub to your Google drive. 

In [ ]:
!git clone https://github.com/TL-System/plato

Get into the `plato` directory:

In [ ]:
!chmod -R ugo+rx $root_path/plato/run
%cd $root_path/plato/

## 2. Connect to the remote

Sign up for a free account in [ngrok](https://ngrok.com) if you haven't, and then use your authentication token and your account password in the following code:

In [ ]:
!pip install colab_ssh --upgrade

from getpass import getpass
ngrok_token = getpass('Your authentication token: ')
password = getpass('The SSH password that you want to set (optional): ')

from colab_ssh import launch_ssh, init_git
launch_ssh(ngrok_token, password)

`launch_ssh` will produce an SSH configuration for you to add to your Visual Studio Code setup, so that you can use **Remote-SSH: Connect to Host...** in Visual Studio Code to connect to this Colab instance. After your SSH connection is setup, you can use your instance just like any other remote virtual machine in the cloud. Detailed steps are:

1. Install the `Remote-SSH: Editing Configuration Files` extension in Visual Studio Code.

2. In Visual Studio Code, click on `View > Command Palette` in the menu (or use `Shift+Command+P`), and type `Remote-SSH: Add New SSH Host...`. It will ask you to enter SSH Connection Command. Enter `root@google_colab_ssh`.

3. Select the SSH configuration file to update, copy the conguration information you get after running the above cell into the selected SSH configuration file. The conguration information should be similar to
```
Host google_colab_ssh
		HostName X.tcp.ngrok.io
		User root
		Port XXXXX
```
Then save this configuration file.

4. Click on `View > Command Palette` (or use `Shift+Command+P`) again and type `Remote-SSH: Connect to Host...`. You should see the host `google_colab_ssh` you just added. Click it and Visual Studio will automatically open a new window for you, and prompt for the SSH password you set.

5. Enter your SSH password you just set and you will be connected to the remote.

6. Open folder `/content/drive/MyDrive/plato/`.

## 3. Set the conda environment for running Plato

Download [Miniconda](https://docs.conda.io/en/latest/miniconda.html#linux-installers) Linux installers (Python 3.8) on your local computer and upload it to your Google drive. For example, I put it under `/content/drive/MyDrive`. 

Open the integrated terminal in Visual Studio Code and use the following command to install Minicoda:

```shell
$ bash /content/drive/MyDrive/Miniconda3-latest-Linux-x86_64.sh
```

When it prompts
```
Miniconda3 will now be installed into this location:
/root/miniconda3

  - Press ENTER to confirm the location
  - Press CTRL-C to abort the installation
  - Or specify a different location below

[/root/miniconda3] >>> 
```

type 
```
/content/drive/MyDrive/miniconda3
```
and hit Enter. Here we install Miniconda on your own drive, instead of on the remote instance, so that all conda environments created by you in the future will also be installed on your drive.


After installation, create a new `conda` environment with Python 3.8 using the command:

```shell
$ conda update conda
$ conda create -n federated python=3.8
$ conda activate federated
```

where `federated` is the preferred name of your new environment. Update any packages, if necessary by typing `y` to proceed.

The next step is to install Plato as a pip package, and torch:

```shell
$ pip install .
$ pip install torch torchvision
```


# Later

When you connect to the remote virtual machine after the first time, run the following cell to avoid getting disconnected first.

In [ ]:
%%javascript
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}setInterval(ClickConnect,60000)

Same old step: produce an SSH configuration and connect to this Colab instance. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
root_path = '/content/drive/My\ Drive'
%cd $root_path
!chmod -R ugo+rx $root_path/ngrok
!chmod -R ugo+rx $root_path/plato/run

In [ ]:
!pip install colab_ssh --upgrade

from getpass import getpass
ngrok_token = getpass('Your authentication token: ')
password = getpass('The SSH password that you want to set (optional): ')

from colab_ssh import launch_ssh, init_git
launch_ssh(ngrok_token, password)

After connecting to the remote instance, run

In [ ]:
f = open("/root/.bashrc", "a")
f.write("""
# >>> conda initialize >>>
# !! Contents within this block are managed by 'conda init' !!
__conda_setup="$('/content/drive/MyDrive/miniconda3/bin/conda' 'shell.bash' 'hook' 2> /dev/null)"
if [ $? -eq 0 ]; then
    eval "$__conda_setup"
else
    if [ -f "/content/drive/MyDrive/miniconda3/etc/profile.d/conda.sh" ]; then
        . "/content/drive/MyDrive/miniconda3/etc/profile.d/conda.sh"
    else
        export PATH="/content/drive/MyDrive/miniconda3/bin:$PATH"
    fi
fi
unset __conda_setup
# <<< conda initialize <<<
alias plato='conda activate federated'
""")
f.close()

!mkdir /root/.conda/
f = open("/root/.conda/environments.txt", "w")
f.write("""/content/drive/MyDrive/miniconda3
/content/drive/MyDrive/miniconda3/envs/federated
""")
f.close()

!chmod -R ugo+rx /content/drive/MyDrive/miniconda3/bin
!chmod -R ugo+rx /content/drive/MyDrive/miniconda3/envs/federated/


Then go back to the integrated terminal in Visual Studio Code and type:

```shell
$ source /root/.bashrc
```

This may take a while. Please wait until you see `(federated)` at the beginning of the next prompt.

And finally, you can activate your environment `federated` to run *Plato*:

```shell
$ plato
```
